In [49]:
import pandas as pd
import string
import re
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import one_hot
# from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

import numpy as np

In [50]:
politifact = pd.read_json(r'./data/politifact.json')
snopes = pd.read_json(r'./data/snopes.json')

In [51]:
# column names
poli_cols = politifact.columns.to_list()
snop_cols = snopes.columns.to_list()

# poli_row_count, poli_col_count = politifact.shape
# snop_row_count, snop_col_count = snopes.shape

In [52]:
common_cols = set(poli_cols) & set(snop_cols)

td = pd.concat([snopes[list(common_cols)], politifact[list(common_cols)]])
td = td.reindex(columns = ['doc', 'claim', 'factchecker', 'url', 'sources', 'topic', 'published' ,'label' ])


In [53]:

td['label'] = td['label'].map({
    'Correct Attribution':0,
    'False':1,
    'Labeled Satire':2,
    'Legend':3,
    'Lost Legend':4,
    'Misattributed':5,
    'Miscaptioned':6,
    'Mixture':7,
    'Mostly False':8,
    'Mostly True':9,
    'Outdated':10,
    'Research In Progress':11,
    'Scam':12,
    'True':13,
    'Unproven':14,
    'barely-true':15,
    'false':1,
    'full-flop':16,
    'half-flip':17,
    'half-true':18,
    'mostly-true':19,
    'no-flip':20,
    'pants-fire':21,
    'true':13}
 )

In [54]:
mt = {
        '\n':" ",
        "'":None,
        '"':None,
        '“':None,
        '”':None,
        '’':None,
        '‘':None
    }
for n in range(10):
    mt[str(n)]=None

for x in string.punctuation:
    mt[x]=None
cols = td.columns.to_list()
cols.remove('label')

# cleaning the first file
for col in tqdm(cols):
    for indx, x in enumerate(td[col]):
        try:
            td.loc[indx,col] = (x.translate(x.maketrans(mt))).lower()
        except:
            if x==None:
                td.loc[indx,col] = ""
            elif type(x) == list:
                p = " ".join(str(z) for z in x)
                td.loc[indx,col] = p.translate(p.maketrans(mt)).lower()
            else:
                print(f"column:{col}\trow:{indx}\tvalue: {x} is invalid\tvalue type: {type(x)}")

100%|█████████████████████████████████████████████████████████████████████████████████████| 7/7 [03:09<00:00, 27.04s/it]


In [55]:
td = td.dropna(subset=['doc','claim','label'])

In [56]:
one_hot_prep = [one_hot(series['claim']+' '+ series['doc'],10000) for _, series in td.iterrows()]

In [57]:
temp = [len(x) for x in one_hot_prep]
print(max(temp))

6955


In [58]:
sent_len = 300
embedded_docs = pad_sequences(one_hot_prep,padding='post',truncating='post',maxlen=sent_len)

In [59]:
sent_len

300

In [60]:
# embedding_vector_fetures = 64
# vocab_size = 5000

# model = Sequential()
# model.add(Embedding(vocab_size,embedding_vector_fetures,input_length= sent_len))
# model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))#LSTM(100))
# model.add(Dense(24, activation = 'softmax'))
# model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
vocab_size=50000
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length= sent_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(22, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 300, 100)          5000000   
                                                                 
 spatial_dropout1d_2 (Spatia  (None, 300, 100)         0         
 lDropout1D)                                                     
                                                                 
 lstm_3 (LSTM)               (None, 100)               80400     
                                                                 
 dense_3 (Dense)             (None, 22)                2222      
                                                                 
Total params: 5,082,622
Trainable params: 5,082,622
Non-trainable params: 0
_________________________________________________________________
None


In [79]:
# x,_ = td.shape

# y = td['label']
# new_y = np.zeros((x,24))

# for i,r in enumerate(y):
#     try:
#         new_y[i][int(r)]=1
#     except:
#         print(i, y[i])
    

In [61]:
final_x = np.array(embedded_docs)
final_y = np.array(td['label'])

In [62]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(final_x, final_y, test_size = 0.2, random_state = 42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(17838, 300) (4460, 300) (17838,) (4460,)


In [64]:
import tensorflow as tf
final_y_train = tf.keras.utils.to_categorical(y_train, num_classes=22)
final_y_test = tf.keras.utils.to_categorical(y_test, num_classes=22)

In [65]:
x_train.shape, final_y_train.shape,x_test.shape,final_y_test.shape
# print(final_y_train.shape, final_y_test.shape)
# print(x_train.shape, x_test.shape)

((17838, 300), (17838, 22), (4460, 300), (4460, 22))

In [66]:
model.fit(x_train, final_y_train, batch_size=32, epochs=10, validation_data=(x_test,final_y_test))

Epoch 1/10
558/558 [==============================] - 115s 200ms/step - loss: 2.1209 - accuracy: 0.2615 - val_loss: 2.0834 - val_accuracy: 0.2661
Epoch 2/10
558/558 [==============================] - 112s 200ms/step - loss: 2.0741 - accuracy: 0.2629 - val_loss: 2.0820 - val_accuracy: 0.2643
Epoch 3/10
558/558 [==============================] - 111s 200ms/step - loss: 2.0212 - accuracy: 0.2785 - val_loss: 2.1236 - val_accuracy: 0.2247
Epoch 4/10
558/558 [==============================] - 112s 200ms/step - loss: 1.9491 - accuracy: 0.2983 - val_loss: 2.1972 - val_accuracy: 0.2049
Epoch 5/10
558/558 [==============================] - 111s 200ms/step - loss: 1.8887 - accuracy: 0.3209 - val_loss: 2.2498 - val_accuracy: 0.2090
Epoch 6/10
558/558 [==============================] - 111s 200ms/step - loss: 1.8405 - accuracy: 0.3239 - val_loss: 2.3091 - val_accuracy: 0.1982
Epoch 7/10
558/558 [==============================] - 112s 200ms/step - loss: 1.7988 - accuracy: 0.3331 - val_loss: 2.3486 -

In [67]:
loss, accuracy = model.evaluate(x_test,final_y_test)
print(accuracy,loss)

140/140 [==============================] - 4s 32ms/step - loss: 2.5255 - accuracy: 0.1888
0.1887892335653305 2.5254507064819336
